In [94]:
import os
import nltk
import tweepy
from dotenv import load_dotenv
import json
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
from datetime import datetime
import re
import cv2
import numpy as np
import pandas as pd
import pickle

In [101]:
import ssl
def set_up_ssl():
    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context

In [102]:
load_dotenv()
TWITTER_BEARER_TOKEN = os.getenv('TWITTER_BEARER_TOKEN')
TWITTER_API_KEY = os.getenv('TWITTER_API_KEY')
TWITTER_API_SECRET_KEY = os.getenv('TWITTER_API_SECRET_KEY')
TWITTER_ACCESS_TOKEN = os.getenv('TWITTER_ACCESS_TOKEN')
TWITTER_ACCESS_TOKEN_SECRET = os.getenv('TWITTER_ACCESS_TOKEN_SECRET')

In [103]:
print("Authenticating to Twitter...")

client = tweepy.Client(bearer_token=TWITTER_BEARER_TOKEN, wait_on_rate_limit=True)
auth = tweepy.OAuthHandler(TWITTER_API_KEY, TWITTER_API_SECRET_KEY)
auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

Authenticating to Twitter...


In [104]:
twitter_russia_sources_rus_usernames = ["@1tvru_news", "@ru_rbc",
                                         "@er_novosti",
                              "@rt_com",
                              "@medvedevrussia", "@kremlinrussia",
                              "@rentvchannel", "@vesti_news", "@kpru"]

twitter_ukraine_sources_rus_usernames = ["@dmitry_gordon", "@SvobodaRadio",
                               "@euronewsru", "@FeyginMark4", "@tvrain", "@teamnavalny"]

twitter_ukraine_sources_ukr_usernames = ["@HromadskeUA", "@tsnua", "@24tvua", "@unian",
                               "@radiosvoboda", "@5channel", "@EspresoTV"]

twitter_ukraine_sources_eng_usernames = ["@mschwirtz", "@KyivIndependent", "@KyivPost",
                               "@mchancecnn", "@fpleitgenCNN", "@Kasparov63",
                               "@ikhurshudyan", "@myroslavapetsa",
                               "@langfittnpr", "@ElBeardsley", "@timkmak"]

https://dev.to/twitterdev/a-comprehensive-guide-for-using-the-twitter-api-v2-using-tweepy-in-python-15d9

In [89]:
def get_user_id_from_username(username):
    user = api.get_user(screen_name=username)
    return user.id

In [105]:
russia_sources_rus = []
ukraine_sources_rus = []
ukraine_sources_ukr = []
ukraine_sources_eng = []

In [107]:
LOAD_CLUSTERS_DATA = True

In [108]:
if not LOAD_CLUSTERS_DATA:
    for username in twitter_russia_sources_rus_usernames:
        russia_sources_rus.append((username, get_user_id_from_username(username)))

    for username in twitter_ukraine_sources_rus_usernames:
        ukraine_sources_rus.append((username, get_user_id_from_username(username)))

    for username in twitter_ukraine_sources_ukr_usernames:
        ukraine_sources_ukr.append((username, get_user_id_from_username(username)))

    for username in twitter_ukraine_sources_eng_usernames:
        ukraine_sources_eng.append((username, get_user_id_from_username(username)))

In [109]:
if not LOAD_CLUSTERS_DATA:
    russia_sources_rus_pickled = pickle.dumps(russia_sources_rus)
    ukraine_sources_rus_pickled = pickle.dumps(ukraine_sources_rus)
    ukraine_sources_ukr_pickled = pickle.dumps(ukraine_sources_ukr)
    ukraine_sources_eng_pickled = pickle.dumps(ukraine_sources_eng)

In [112]:
CLUSTERS_SERIALIZATION_DIR = "data_clusters/"

In [110]:
if not LOAD_CLUSTERS_DATA:
    print("Writing pickled data to a file...")

    with open(CLUSTERS_SERIALIZATION_DIR + 'russia_sources_rus_pickled.pickle', 'wb') as f:
        f.write(russia_sources_rus_pickled)

    with open(CLUSTERS_SERIALIZATION_DIR + 'ukraine_sources_rus_pickled.pickle', 'wb') as f:
        f.write(ukraine_sources_rus_pickled)

    with open(CLUSTERS_SERIALIZATION_DIR + 'ukraine_sources_ukr_pickled.pickle', 'wb') as f:
        f.write(ukraine_sources_ukr_pickled)

    with open(CLUSTERS_SERIALIZATION_DIR + 'ukraine_sources_eng_pickled.pickle','wb') as f:
        f.write(ukraine_sources_eng_pickled)

In [113]:
if LOAD_CLUSTERS_DATA:
    with open(CLUSTERS_SERIALIZATION_DIR + 'russia_sources_rus_pickled.pickle',\
            'rb') as f:
        russia_sources_rus = pickle.load(f)

    with open(CLUSTERS_SERIALIZATION_DIR + 'ukraine_sources_rus_pickled' \
            '.pickle', 'rb') as f:
        ukraine_sources_rus = pickle.load(f)

    with open(CLUSTERS_SERIALIZATION_DIR + 'ukraine_sources_ukr_pickled' \
            '.pickle', 'rb') as f:
        ukraine_sources_ukr = pickle.load(f)

    with open(CLUSTERS_SERIALIZATION_DIR + 'ukraine_sources_eng_pickled' \
            '.pickle', 'rb') as f:
        ukraine_sources_eng = pickle.load(f)

In [41]:
def get_user_followers(username, user_id):
    user_followers = {}
    followers = []
    followers_paginator = tweepy.Paginator(client.get_users_followers, id =
    user_id, max_results = 10, limit = 2).flatten()
    for follower in followers_paginator:
        followers.append(follower)
    user_followers[(username, user_id)] = followers
    return user_followers

In [42]:
followers = get_user_followers("@minregion_ua", 3333475643)

Rate limit exceeded. Sleeping for 118 seconds.


In [50]:
# the screen_name of the targeted user
screen_name = "geeksforgeeks"

# printing the latest 20 followers of the user
for follower in api.get_followers(screen_name):
    print(follower.screen_name)

TypeError: API.get_followers() takes 1 positional argument but 2 were given

In [49]:
print(len(followers))

1


In [58]:
print(len(followers[('@minregion_ua', 3333475643)]))
print(followers[('@minregion_ua', 3333475643)])

20
[<User id=1596438068617269248 name=Ігор Зірчак username=IGOR2425888>, <User id=1596233105844707328 name=Valeryy Spetsteh username=spetstehavto>, <User id=2541291170 name=Igor Cvetkovski username=igorcvetkovski7>, <User id=1539285531359596544 name=Nik Cool username=nikcool175>, <User id=2541664184 name=UKR Embassy in HUN username=UKRinHUN>, <User id=2549703642 name=якесь кріативне безглуздя username=pkconline1>, <User id=1567116049002233857 name=nn nn username=nnnn60913189>, <User id=1553414616281227265 name=Влад Казбан username=patriot_Ukr_>, <User id=109796666 name=Klaas Pompstra 🇺🇦 username=KlaasPompstra>, <User id=1502471574871441412 name=ricecake username=risovoyetesto>, <User id=1565987672329490432 name=Dmitry Olefirov username=MaestroRew>, <User id=2687387750 name=Роман username=chipoli2>, <User id=1534639698341154817 name=Гордей username=Gordej23505730>, <User id=1259664191415357443 name=medco.at.ua Asia username=AsiaOvertime>, <User id=1071629801004380160 name=wUAw! username

In [71]:
print(type(followers[('@minregion_ua', 3333475643)][0]))
print(followers[('@minregion_ua', 3333475643)][0].name)
print(followers[('@minregion_ua', 3333475643)][0].id)

<class 'tweepy.user.User'>
Ігор Зірчак
1596438068617269248


In [86]:
# cluster center : followers
# each cluster has a dataframe of followers with username and id field

clusters = {}

for follower in followers:
    clusters[follower] = pd.DataFrame([followers[follower][0].name,
                                       followers[follower][0].id],
                                            columns=['Username','Id'])

ValueError: Shape of passed values is (2, 1), indices imply (2, 2)

In [85]:
# write the dataframes to a file as csv

print(clusters)

{}
